**Specifying the TensorFlow version for COLAB**

In [0]:
%tensorflow_version 1.x

**Connect To Drive (For COLAB)**

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Install Keras-Contrib

In [0]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-7_vk9ext
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-7_vk9ext
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=026b28009a393b4615253ce7594048c5801642dbcee09d030b3e3948f0535e05
  Stored in directory: /tmp/pip-ephem-wheel-cache-halybi1k/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


Import All Libraries

In [0]:
#from keras.datasets import mnist
import keras
#from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
#from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import concatenate, Concatenate, UpSampling2D, Conv2D, Conv2DTranspose, MaxPooling2D, Activation, Flatten, Dense, Input, BatchNormalization, Dropout, Reshape
#from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
#from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import sys
#from data_loader import DataLoader

import h5py
import os
import matplotlib.pyplot as plt
import cv2
import keras
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from __future__ import print_function, division
import scipy
import pickle

Using TensorFlow backend.


**Load DataSet and Split into train and test**

In [0]:
base_path = '/content/drive/My Drive/DataSets/Brain/processedBrats_FullBrain/V3_Brats2017'
with h5py.File(os.path.join(base_path, 'v2_withlabel01_imgs_without_masks-imgs_with_masks-imgs_masks_zscore_norm__float32.hdf5'), 'r') as f:
  imgs_without_masks = f['imgs_without_masks'].value
  imgs_with_masks = f['imgs_with_masks'].value
  imgs_masks = f['imgs_masks'].value
print('DataSet Loaded Successfully')

imgs_without_masks = imgs_without_masks.reshape(-1, imgs_without_masks.shape[1], imgs_without_masks.shape[2], 1)
imgs_with_masks = imgs_with_masks.reshape(-1, imgs_with_masks.shape[1], imgs_with_masks.shape[2], 1)
imgs_masks = imgs_masks.reshape(-1, imgs_masks.shape[1], imgs_masks.shape[2], 1)
#####

test_start = 7
test_size = 10#set test size. 

imgs_with_masks_test = imgs_with_masks[test_start:test_start+test_size]
imgs_with_masks = np.delete(imgs_with_masks, np.arange(test_start, test_start+test_size), axis=0)

imgs_masks_test = imgs_masks[test_start:test_start+test_size]
imgs_masks = np.delete(imgs_masks, np.arange(test_start, test_start+test_size), axis=0)

test_start = 0
test_size = 10#set test size.
imgs_without_masks_test = imgs_without_masks[test_start:test_start+test_size]
imgs_without_masks = np.delete(imgs_without_masks, np.arange(test_start, test_start+test_size), axis=0)

print('imgs_with_masks_test', imgs_with_masks_test.shape)
print('imgs_with_masks', imgs_with_masks.shape)
print('imgs_masks_test', imgs_masks_test.shape)
print('imgs_masks', imgs_masks.shape)
print('imgs_without_masks_test', imgs_without_masks_test.shape)
print('imgs_without_masks', imgs_without_masks.shape)

DataSet Loaded Successfully
imgs_with_masks_test (10, 256, 256, 1)
imgs_with_masks (2701, 256, 256, 1)
imgs_masks_test (10, 256, 256, 1)
imgs_masks (2701, 256, 256, 1)
imgs_without_masks_test (10, 256, 256, 1)
imgs_without_masks (453, 256, 256, 1)


**Cycle GAN** ***(UNET)***

In [0]:
def overlayMaskonImg(img, msk):
    image = (((img - np.min(img))/(np.max(img) - np.min(img)))*255).astype('uint8')
    mask = (((msk - np.min(msk))/(np.max(msk) - np.min(msk)))*255).astype('uint8')
    mask = np.where(mask>0,255,0).astype('uint8')
    image = cv2.merge([image, image, image])
    mask = cv2.merge([mask, np.zeros(mask.shape).astype('uint8'), np.zeros(mask.shape).astype('uint8')])
    indices = np.where(mask==255)
    mask[indices[0], indices[1], :] = [255, 0, 0]    
    
    alpha = 0.6
    #cv2.addWeighted(mask, alpha, image, 1 - alpha, 0, image)
    cv2.addWeighted(mask, alpha, image, 1, 0, image)
    
    return image

def getHeatMap(bimg, msk):
  image = bimg
  image = (image - np.min(image))/(np.max(image) - np.min(image))
  image = (image*255).astype('uint8')
  mask = msk
  cmap = plt.get_cmap('jet')
  rgba_img = cmap(mask)
  rgb_img = np.delete(rgba_img, 3, 2)

  alpha = 0.4
  output = cv2.merge([image, image, image])#image.copy().astype('uint8')
  overlay = (rgb_img.copy()*255).astype('uint8')
  foutput = cv2.addWeighted(overlay, alpha, output, 1 - alpha, 0, output)

  return foutput
  
def getbinaryMaskFromHeatMap(heatmap):
  img1 = heatmap.copy()
  image_hsv = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
  h, _, _ = cv2.split(image_hsv)
  mask = cv2.inRange(h, 95, 1000)
  blob = cv2.bitwise_and(img1, img1, mask=mask)

  foutput = np.where(cv2.cvtColor(blob, cv2.COLOR_BGR2GRAY) > 10 ,1, 0)
  return blob, foutput

In [0]:
class CX_GAN():
    def __init__(self):
        self.img_rows = 256
        self.img_cols = 256
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 32
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0                    # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle    # Identity loss

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        fake_A = keras.layers.Add()([fake_A, img_B])
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_A = keras.layers.Add()([reconstr_A, fake_B])
        reconstr_B = self.g_AB(fake_A)
        # Identity mapping of images
        img_A_id = self.g_BA(img_A)
        img_A_id = keras.layers.Add()([img_A_id, img_A])
        img_B_id = self.g_AB(img_B)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        reconstr_A, reconstr_B,
                                        img_A_id, img_B_id ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                            loss_weights=[  1, 1,
                                            self.lambda_cycle, self.lambda_cycle,
                                            self.lambda_id, self.lambda_id ],
                            optimizer=optimizer)

    def random_crop(self, image, imgs_count):
      cropped_image = tf.image.random_crop(
      image, size=[imgs_count, self.img_cols, self.img_rows, self.channels])
      return cropped_image

    def random_jitter(self, image):
        # resizing to 286 x 286 x 3
        imgs_count = image.shape[0]
        image = tf.image.resize_images(image, [self.img_cols, self.img_rows],
                                method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

        # randomly cropping to 256 x 256 x 3
        image = self.random_crop(image, imgs_count)

        # random mirroring
        with tf.Session() as sess:
          image = tf.image.random_flip_left_right(image).eval()#self.sess.run(tf.image.random_flip_left_right(image))#tf.image.random_flip_left_right(image).eval()
          
        return image

    def getRandomindexes(self, end, sampleSize, _replace = False):
        return np.random.choice(np.asarray(range(end)), sampleSize, replace = _replace)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)

        # Upsampling
        u1 = deconv2d(d4, d3, self.gf*4)
        u2 = deconv2d(u1, d2, self.gf*2)
        u3 = deconv2d(u2, d1, self.gf)

        u4 = UpSampling2D(size=2)(u3)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1, padding='same', activation='tanh')(u4)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    def train(self, epochs, ModelSavePath, model_save_intervel=5, restoreFrom = None, batch_size=1, sample_interval=50):
        basePath_ModelSave = ModelSavePath
        run_ModelSave = ''
        epoch_start = 0
        if restoreFrom is None:
          if not os.path.exists(basePath_ModelSave):
            os.mkdir(basePath_ModelSave)

          if len(os.listdir(basePath_ModelSave)) == 0:
            vName = 0
          else:
            vName = [int(x[-1]) for x in sorted(os.listdir(basePath_ModelSave))]
          run_ModelSave = 'run'+str(np.asarray(vName).max()+1)
          os.mkdir(os.path.join(basePath_ModelSave, run_ModelSave))
          os.mkdir(os.path.join(basePath_ModelSave, run_ModelSave, str('Qualitative_Results')))
          with open(os.path.join(basePath_ModelSave, run_ModelSave, 'np_random_state.pickle'), 'wb') as f:
            pickle.dump(np.random.get_state(), f)
        else:
          epoch_start = np.asarray([int(x[6:]) for x in sorted(os.listdir(os.path.join(basePath_ModelSave, restore_From))) if x[:6] == 'epoch_']).max()
          run_ModelSave = restoreFrom
          self.loadWeights(os.path.join(basePath_ModelSave, run_ModelSave,'epoch_'+str(epoch_start)), '')
          #print(os.path.join(basePath_ModelSave, run_ModelSave))
        log_file = open(os.path.join(basePath_ModelSave, run_ModelSave,"log.txt"),"a+")
        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)
        imgs_len = int(np.where(imgs_with_masks.shape[0] > imgs_without_masks.shape[0], imgs_with_masks.shape[0], imgs_without_masks.shape[0]))
        for epoch in range(epoch_start+1, epochs+1):
            #for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):
            rndm_ind_imgs_with_msks = self.getRandomindexes(len(imgs_with_masks), imgs_len, _replace = (imgs_with_masks.shape[0]<imgs_without_masks.shape[0]))
            rndm_ind_imgs_without_msks = self.getRandomindexes(len(imgs_without_masks), imgs_len, _replace = (imgs_with_masks.shape[0]>imgs_without_masks.shape[0]))
            imgs_with_masks_Augmented = self.random_jitter(imgs_with_masks).reshape(-1, self.img_rows, self.img_cols, self.channels)
            imgs_without_masks_Augmented = self.random_jitter(imgs_without_masks).reshape(-1, self.img_rows, self.img_cols, self.channels)
            for batch_i in range(imgs_len):#Use batch size as 1
                imgs_A = (imgs_with_masks_Augmented[rndm_ind_imgs_with_msks[batch_i]]).reshape(1, self.img_rows, self.img_cols, self.channels)#self.random_jitter(imgs_with_masks[rndm_ind_imgs_with_msks[batch_i]]).reshape(-1, self.img_rows, self.img_cols, self.channels)#imgs_with_masks[(batch_i)*batch_size:(batch_i+1)*batch_size]
                imgs_B = (imgs_without_masks_Augmented[rndm_ind_imgs_without_msks[batch_i]]).reshape(1, self.img_rows, self.img_cols, self.channels)#self.random_jitter(imgs_without_masks[rndm_ind_imgs_without_msks[batch_i]]).reshape(-1, self.img_rows, self.img_cols, self.channels)#imgs_without_masks[(batch_i)*batch_size:(batch_i+1)*batch_size]
                #print('imgs_A.shape', imgs_A.shape)
                #print('imgs_B.shape', imgs_B.shape)
                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)
                fake_A = np.add(fake_A, imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)


                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B],
                                                        [valid, valid,
                                                        imgs_A, imgs_B,
                                                        imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time

                # Plot the progress
                print ("\r[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                                                                        % ( epoch, epochs,
                                                                            batch_i+1, imgs_len//batch_size,
                                                                            d_loss[0], 100*d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time), end = '')
                
            log_file.write("[Epoch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s \n" \
                                                                        % ( epoch, epochs,
                                                                            d_loss[0], 100*d_loss[1],
                                                                            g_loss[0],
                                                                            np.mean(g_loss[1:3]),
                                                                            np.mean(g_loss[3:5]),
                                                                            np.mean(g_loss[5:6]),
                                                                            elapsed_time))
            log_file.flush()

                # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
              self.sample_images_Edited(epoch, batch_i, img_save_path=os.path.join(basePath_ModelSave, run_ModelSave, str('Qualitative_Results'),str(epoch)+'.png'))
              #self.sample_images(epoch, batch_i)
              #self.sample_images_Train(1, 180)

            #Save Model and Weights
            if epoch%model_save_intervel == 0:
              base_dir = os.path.join(basePath_ModelSave, run_ModelSave, 'epoch_'+str(epoch))
              #pre_name = 'cycleGANWithoutIdentity_'
              pre_name = 'cycleGANIdentityAndUniformNorm_0_1_'
              if not os.path.exists(base_dir):
                os.makedirs(base_dir)

              #Save Discriminator 1
              model_json = gan.d_A.to_json()
              with open(os.path.join(base_dir, pre_name+"d_A.json"), "w") as json_file:
                  json_file.write(model_json)
              # serialize weights to HDF5
              gan.d_A.save_weights(os.path.join(base_dir, pre_name+"d_A.h5"))
              #print("Saved model to disk")

              #Save Discriminator 2
              model_json = gan.d_B.to_json()
              with open(os.path.join(base_dir, pre_name+"d_B.json"), "w") as json_file:
                  json_file.write(model_json)
              # serialize weights to HDF5
              gan.d_B.save_weights(os.path.join(base_dir, pre_name+"d_B.h5"))
              #print("Saved model to disk")

              #Save Generator AB
              model_json = gan.g_AB.to_json()
              with open(os.path.join(base_dir, pre_name+"g_AB.json"), "w") as json_file:
                  json_file.write(model_json)
              # serialize weights to HDF5
              gan.g_AB.save_weights(os.path.join(base_dir, pre_name+"g_AB.h5"))
              #print("Saved model to disk")

              #Save Generator BA
              model_json = gan.g_BA.to_json()
              with open(os.path.join(base_dir, pre_name+"g_BA.json"), "w") as json_file:
                  json_file.write(model_json)
              # serialize weights to HDF5
              gan.g_BA.save_weights(os.path.join(base_dir, pre_name+"g_BA.h5"))
              #print("Saved model to disk")

            #End Save Model and Weights
        log_file.close()

    def sample_images(self, epoch, batch_i):
        r, c = 2, 4

        #imgs_A = imgs_with_masks[-1:]
        imgs_A = imgs_with_masks_test[-1:]
        imgs_B = imgs_without_masks[-1:]

        # Demo (for GIF)
        #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(np.add(fake_A, imgs_B))
        global map0
        map0 = reconstr_A
        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, np.add(reconstr_A, fake_B), imgs_B, fake_A, np.add(fake_A, imgs_B), reconstr_B])
        
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = [['Original', 'Translated', 'Map', 'Reconstructed'], ['Original', 'Map', 'Translated', 'Reconstructed']]
        fig, axs = plt.subplots(r, c, figsize= (15, 10))
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt].squeeze(), cmap='gray')
                axs[i, j].set_title(titles[i][j])
                axs[i,j].axis('off')
                cnt += 1
        plt.show()
        plt.close()

    def sample_images_Only_Infected(self, Indx=0):
        r, c = 1, 6

        #imgs_A = imgs_with_masks[-1:]
        imgs_A = imgs_with_masks_test[Indx:Indx+1]
        #imgs_B = imgs_without_masks_test[0:1]
        img_msk = imgs_masks_test[Indx:Indx+1]
 
        # Demo (for GIF)
        #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        #fake_A = self.g_AB.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        #reconstr_B = self.g_BA.predict(np.add(fake_A, imgs_B))
        #reconstr_B = self.g_BA.predict(fake_A)
        #global map0
        #map0 = reconstr_A
        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A])
        
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = [['Input (Class: 1)', 'Translated (Class: 0)', 'Map',]]
        fig, axs = plt.subplots(r, c, figsize= (21, 6))
        cnt = 0
        for i in range(r):
            for j in range(c-3):
                # print(cnt)
                # print(gen_imgsshape)
                axs[j].imshow(gen_imgs[cnt].squeeze(), cmap='gray')
                axs[j].set_title(titles[i][j])
                axs[j].axis('off')
                cnt += 1
            if(i == 0):
              hmap = getHeatMap(imgs_A.squeeze(), reconstr_A.squeeze()).squeeze()
              axs[4].imshow(hmap)
              _, binarymsk = getbinaryMaskFromHeatMap(hmap)
              #axs[4].imshow(colormsk, cmap='gray')
              axs[3].imshow(binarymsk, cmap='gray')
              axs[5].imshow(overlayMaskonImg(imgs_A.astype('float64').reshape(self.img_shape).squeeze(), img_msk.astype('float64').reshape(self.img_shape)))
            else:
              hmap = getHeatMap(imgs_B.squeeze(), reconstr_B.squeeze()).squeeze()
              axs[54].imshow(hmap)
              _, binarymsk = getbinaryMaskFromHeatMap(hmap)
              #axs[4].imshow(colormsk, cmap='gray')
              axs[3].imshow(binarymsk, cmap='gray')
              axs[5].imshow(imgs_B.squeeze(), cmap='gray')
            axs[4].set_title('HeatMap')
            axs[4].axis('off')
            #axs[4].set_title('Color Mask')
            #axs[4].axis('off')
            axs[3].set_title('Binary Mask')
            axs[3].axis('off')
            axs[5].set_title('GroundTruth')
            axs[5].axis('off')
        hmap = getHeatMap(imgs_A.squeeze(), reconstr_A.squeeze()).squeeze()
        _, binarymsk = getbinaryMaskFromHeatMap(hmap)
        plt.savefig('/content/drive/My Drive/ResultsPreparation/synopsisResults_For_Qualitative/Proposed Approach/'+'proposed-GAN'+str(Indx)+'.png')#os.path.join(img_save_path))
        
        plt.imsave('/content/drive/My Drive/ResultsPreparation/synopsisResults_For_Quantitative/Proposed Approach/'+'proposed-GAN'+str(Indx)+'_original.png', img_msk[i].squeeze(), cmap='gray')
        plt.imsave('/content/drive/My Drive/ResultsPreparation/synopsisResults_For_Quantitative/Proposed Approach/'+'proposed-GAN'+str(Indx)+'_predicted.png', binarymsk.squeeze(), cmap='gray')
  
        plt.show()
        plt.close()

    def sample_images_Advanced(self, epoch, batch_i, img_save_path):
        r, c = 2, 8

        #imgs_A = imgs_with_masks[-1:]
        imgs_A = imgs_with_masks_test[0:1]
        imgs_B = imgs_without_masks_test[0:1]
        img_msk = imgs_masks_test[0:1]
 
        # Demo (for GIF)
        #imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        #imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_AB.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        #reconstr_B = self.g_BA.predict(np.add(fake_A, imgs_B))
        reconstr_B = self.g_BA.predict(fake_A)
        global map0
        map0 = reconstr_A
        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, np.add(reconstr_A, fake_B), imgs_B, fake_A, reconstr_B, np.add(fake_A, imgs_B)])
        
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = [['Original', 'Translated', 'Map', 'Reconstructed'], ['Original', 'Translated', 'Map', 'Reconstructed']]
        fig, axs = plt.subplots(r, c, figsize= (21, 6))
        cnt = 0
        for i in range(r):
            for j in range(c-4):
                axs[i,j].imshow(gen_imgs[cnt].squeeze(), cmap='gray')
                axs[i, j].set_title(titles[i][j])
                axs[i,j].axis('off')
                cnt += 1
            if(i == 0):
              hmap = getHeatMap(imgs_A.squeeze(), reconstr_A.squeeze()).squeeze()
              axs[i,4].imshow(hmap)
              colormsk, binarymsk = getbinaryMaskFromHeatMap(hmap)
              axs[i,5].imshow(colormsk, cmap='gray')
              axs[i,6].imshow(binarymsk, cmap='gray')
              axs[i,7].imshow(overlayMaskonImg(imgs_A.astype('float64').reshape(self.img_shape).squeeze(), img_msk.astype('float64').reshape(self.img_shape)))
            else:
              hmap = getHeatMap(imgs_B.squeeze(), reconstr_B.squeeze()).squeeze()
              axs[i,4].imshow(hmap)
              colormsk, binarymsk = getbinaryMaskFromHeatMap(hmap)
              axs[i,5].imshow(colormsk, cmap='gray')
              axs[i,6].imshow(binarymsk, cmap='gray')
              axs[i,7].imshow(imgs_B.squeeze(), cmap='gray')
            axs[i,4].set_title('HeatMap')
            axs[i,4].axis('off')
            axs[i,5].set_title('Color Mask')
            axs[i,5].axis('off')
            axs[i,6].set_title('Binary Mask')
            axs[i,6].axis('off')
            axs[i,7].set_title('GroundTruth')
            axs[i,7].axis('off')
            
        #fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        fig.savefig(os.path.join(img_save_path))
        plt.show()
        plt.close()

    def loadWeights(self, path, pre_name):
      pre_name = os.path.join(path, pre_name)
      self.d_A.load_weights(pre_name+"d_A.h5")
      self.d_B.load_weights(pre_name+"d_B.h5")
      self.g_AB.load_weights(pre_name+"g_AB.h5")
      self.g_BA.load_weights(pre_name+"g_BA.h5")
      print('Loading Weights From', path)


    def sample_images_Train(self, ref0, ref1):
        print('on train set::')
        r, c = 2, 4

        imgs_A = imgs_with_masks_test[ref0:ref0+1]
        imgs_B = imgs_without_masks[ref1:ref1+1]

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(np.add(fake_A, imgs_B))
        global map1
        map1 = reconstr_A
        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, np.add(reconstr_A, fake_B), imgs_B, fake_A, np.add(fake_A, imgs_B), reconstr_B])
        
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = [['Original', 'Translated', 'Map', 'Reconstructed'], ['Original', 'Map', 'Translated', 'Reconstructed']]
        fig, axs = plt.subplots(r, c, figsize= (15, 10))
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt].squeeze(), cmap='gray')
                axs[i, j].set_title(titles[i][j])
                axs[i,j].axis('off')
                cnt += 1
        plt.show()
        plt.close()

In [0]:
gan = CycleGAN()

In [0]:
#restore_From = None ##Set run id
#gan.train(epochs=20, ModelSavePath='SET_PATH', restoreFrom = restore_From, model_save_intervel=1, batch_size=1, sample_interval=1)

In [0]:
#Load weights from specific epoch of saved model in each run
#gan.loadWeights('SPECIFY_PATH','')